In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [2]:
df_train = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
df_test = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")

df_submission = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip")


In [3]:
df_train.head()
#df_test.head(2)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
train_data = df_train['comment_text']
test_data = df_test['comment_text']
all_data = pd.concat([train_data, test_data])

In [5]:
#all_data

In [6]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [7]:
def run_classifier(data, features,  diff_columns, clf):
    kdict = {}
    kdict['class'] = []
    kdict['score_mean'] = []
    kdict['score_std'] = []
    kdict['scores'] = []

    for i in diff_columns:
        labels = data[i]
        res = cross_val_score(clf, features, labels, cv=10, scoring='roc_auc')

        kdict['class'].append(i)
        kdict['score_mean'].append(np.mean(res))
        kdict['score_std'].append(np.std(res))
        kdict['scores'].append(res)

    return pd.DataFrame.from_dict(kdict)

In [8]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, GRU, Embedding, Dropout, Activation,Bidirectional, GlobalMaxPool1D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

from sklearn.model_selection import train_test_split

In [9]:
EMBEDDING_FILE='../input/glove6b50dtxt/glove.6B.50d.txt'

embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

In [10]:
train = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [11]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X_t, y, test_size=.2, shuffle=True)

In [13]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [14]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


(0.020940498, 0.6441043)

In [15]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [16]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(GRU(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 50)           1000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 100)          30600     
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 306   

In [18]:
model.fit(x_train, y_train, batch_size=32, epochs=4, validation_data=(x_test, y_test), shuffle=True)

Epoch 1/4
3990/3990 [==============================] - 631s 158ms/step - loss: 0.0788 - accuracy: 0.8653 - val_loss: 0.0582 - val_accuracy: 0.9942
Epoch 2/4
3990/3990 [==============================] - 644s 161ms/step - loss: 0.0595 - accuracy: 0.9602 - val_loss: 0.0545 - val_accuracy: 0.9941
Epoch 3/4
3990/3990 [==============================] - 650s 163ms/step - loss: 0.0558 - accuracy: 0.9671 - val_loss: 0.0515 - val_accuracy: 0.9936
Epoch 4/4
3990/3990 [==============================] - 659s 165ms/step - loss: 0.0530 - accuracy: 0.9662 - val_loss: 0.0501 - val_accuracy: 0.9941


In [19]:
model.evaluate(x_test, y_test)

998/998 [==============================] - 25s 25ms/step - loss: 0.0501 - accuracy: 0.9941


[0.050090521574020386, 0.9941406846046448]

In [20]:
ypred = model.predict(X_te)

In [21]:
ypred[0]

array([0.9915196 , 0.38328177, 0.9498545 , 0.04390082, 0.87182117,
       0.18093222], dtype=float32)

In [22]:
df_submission = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')

In [23]:
df_submission.head(1)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5


In [24]:
df_submission['toxic'] = ypred[:,0]
df_submission['severe_toxic'] = ypred[:,1]
df_submission['obscene'] = ypred[:,2]
df_submission['threat'] = ypred[:,3]
df_submission['insult'] = ypred[:,4]
df_submission['identity_hate'] = ypred[:,5]

In [25]:
df_submission.head(1)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.99152,0.383282,0.949854,0.043901,0.871821,0.180932


In [26]:
df_submission.to_csv("submission_toxic_comment.csv", index=False)